In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Tutorial de Modelos de Media Ponderada o MA.

En este tutorial realizaremos varios ejercicios de los modelos tipo MA en las series univariadas. Compobaremos sus propiedades básicas estadísticas. Finalmente, realizarás un ejercicio práctico.

**Contenido**
1. [Datos y Modelamiento:](#1)
    1. [Importación de librerías requeridas](#2)
    1. [Modelo de Regresión](#3)
    1. [Graficas](#4)
    1. [Prediccion ](#5)
1. [Ejercicio](#6)
    1. [Modelo en bruto](#7)
1. [Model checking](#9)
    1. [Grafica de los residuales](#10)
    1. [Prediccion](#11)
1. [Tabla Comparativa](#12)

   
    

## Ejemplo MA

<a id="1"></a> <br>
## Datos y Modelamiento

<a id="2"></a> <br>
### Importación de librerías requeridas

In [6]:
library(TSstudio)
set.seed(12345)

In [7]:
## Primeramente, simulemos un proceso tipo MA(2) con 500 puntos.
ma <- arima.sim(model = list(order = c(0, 0, 2), ma = c(0.2,0.6)), n = 500) 
ts_plot(ma, title = "Simulación Proceso MA(2)", Ytitle = "rt", Xtitle = "Index")

Como podemos ver, los procesos MA siempre son estacionarios, a diferencia de los procesos AR(p), los cuales requieren en ocasiones transformaciones para poderlos modelar.

In [8]:
#Aplicamos la prueba de ADF:
library(tseries)
adf.test(ma)

In [9]:
par(mfrow=c(1,2))
acf(ma)
pacf(ma)

<a id="3"></a> <br>
## Modelo de Regresión

In [10]:
#aplicamos un modelo de regresión, dados los resutlados en la ACF
md_ma <- arima(ma, order = c(0,0,2), method = "ML")
md_ma

<a id="4"></a> <br>
### Muestra la gráfica de los residuales

In [11]:
tsdiag(md_ma)

In [12]:
str(md_ma)
md_ma

In [13]:
sigma2 =md_ma$sigma
sigma2
theta1=md_ma$coef[1]
theta1
theta2=md_ma$coef[2]
theta2
theta0=md_ma$coef[3]
theta0

In [14]:
#Determinamos varianza y desv stad:
var_ma = (sigma2)*(1+theta1^2+ theta2^2)
var_ma
sigma=var_ma^0.5
sigma

In [10]:
media=theta0
media

<a id="5"></a> <br>
### Prediccion 

Si generamos una predicción, observemos los valores futuros, donde el valor de la media es igual a la ordenada al origen que se logra h+1 pasos adelante, igualmente para el caso de la desviación estándar, cuyo valor se tiene h+1 pasos adelante.

In [15]:
predict(md_ma,5)

<a id="6"></a> <br>
## Ejercicio 

In [16]:
da=read.table("../input/ibm-ma/ibm_ma.txt",header=T)
head(da)

#### Selecciona la columna de ewrtn (rendimiento simple del índice CRSP Equally weighed)


In [18]:
r1=da$ewrtn

In [19]:
par(mfrow=c(1,2))
acf(r1)
pacf(r1)
length(r1)

<a id="7"></a> <br>
###  Modelo en bruto

In [20]:
#Vamos a modelar esta serie como un MA únicamente. Si observamos la ACF, existen lags significativos hasta el 9, aunque intermedios no lo sean.
m1=arima(r1,order=c(0,0,9))
m1

In [21]:
#Generamos un modelo restrictivo, eliminando los rezagos del ruido: 2,4,5,6,7 y 8
m2=arima(r1,order=c(0,0,9),fixed=c(NA,0,NA,0,0,0,0,0,NA,NA))
m2

<a id="9"></a> <br>
### Model checking

In [22]:
#Revisamos residuales
Box.test(m2$residuals,lag=12,type='Ljung')

In [23]:
#Igualmente, podemos revisar mediante el ajuste a lso grados de libertad:
pv=1-pchisq(17.604,9)  
pv
#Donde obtenemos un modelo no satisfactorio al 5%, pero sí al 10%.

Se pasa por su cercanía a 5%

<a id="10"></a> <br>
### Muestra la grafica de los residuales

In [24]:
tsdiag(m2)

<a id="10a"></a> <br>
### Partición y Modelación en datos de entrenamiento

In [25]:
#Ya que el modelo es un ma(9), es coherente pensar en un pronóstico a 9 o 10 días, por ello, reservaremos la prueba con dicha longitud.
length(r1)

In [26]:
### Modelo de regresión a sección de entrenamiento
mm2=arima(r1[1:(length(r1)-10)],order=c(0,0,9),fixed=c(NA,0,NA,0,0,0,0,0,NA,NA))
mm2

<a id="11"></a> <br>
### Prediccion

Hacemos la predicción del modelo

In [27]:
predict=predict(mm2,10) #se reservaron 10 días
predict

<a id="12"></a> <br>
### Revisión valores pronósticos con propiedades del modelo


In [28]:
#Podemos generar una tabla con los valores actuales, pronosticados y los de su error
actual = r1[987:996] #con los datos que se dejaron de prueba
predic=predict$pred
std_error=predict$se
resultados=data.frame(actual, predic, std_error)
resultados

In [29]:
sigma2 =mm2$sigma
sigma2
theta1=mm2$coef[1]
theta1
theta3=mm2$coef[3]
theta3
theta9=mm2$coef[9]
theta9
theta0=mm2$coef[10]
theta0

In [37]:
media=theta0 #se logra evidenciar el valor de media y en el décimo pronóstico 
media
#sigma = sigma2
#sigma

In [39]:
#Determinamos varianza y desv stad:
var_ma = (sigma2)*(1+theta1^2+ theta3^2+ theta9^2)
#var_ma
sigma=var_ma^0.5
sigma #std_err

Vemos que los valores pronosticados son los mismos que se calculan mediante las propiedades del proceso MA.